# COMP5328 Assignment 2 Loss function correction

# Data pre-processing

In [1]:
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

#------------------------------#
# loaddta(): load data
# standard(): normalise data
#------------------------------#

def loaddata():
    dataset_mnist = np.load('mnist_dataset.npz')
    dataset_cifar = np.load('cifar_dataset.npz')

    Xtr_mnist = dataset_mnist['Xtr']
    Str_mnist = dataset_mnist['Str']
    Xts_mnist = dataset_mnist['Xts']
    Yts_mnist = dataset_mnist['Yts']

    Xtr_cifar = dataset_cifar['Xtr']
    Str_cifar = dataset_cifar['Str']
    Xts_cifar = dataset_cifar['Xts']
    Yts_cifar = dataset_cifar['Yts']

    return Xtr_mnist, Str_mnist, Xts_mnist, Yts_mnist, Xtr_cifar, Str_cifar, Xts_cifar, Yts_cifar


def standard(x_train, x_test):
    '''
    Note:
         Use the std and mean from the training data and implement
         it to the validation data and test data
    RETRUN:
        Flattened and standard(Z-score normalization) training data
        The features size is 3072 or 784
    '''
    std = np.std(x_train, keepdims=True)
    mean = np.mean(x_train, keepdims=True)
    x_train = (x_train-mean)/std
    x_test = (x_test-mean)/std
    return x_train, x_test


def min_max_nor(x_train, x_test, TYPE='mnist'):
    scaler = MinMaxScaler()
    scaler = scaler.fit(x_train)
    x_train = scaler.transform(x_train)
    x_test = scaler.transform(x_test)
    # if TYPE == 'mnist':
    #     return x_train.reshape(28, 28), x_test.reshape(28, 28)
    # else:
    #     return x_train.reshape(32, 32, 3), x_test(32, 32, 3)
    return x_train, x_test


#============#
# split_data
#============#

def split_data(x_train, y_train, part=0.8):
    '''
    Shuffle then data and then split the data accroding to the part parameters
    '''
    # shuffle the data
    order = np.argsort(np.random.random(y_train.shape[0]))
    x_train = x_train[order]
    y_train = y_train[order]

    # split data to train data and validation data
    num_total_train = len(x_train)
    split_index = int(num_total_train*part)

    # index data
    x_vali = x_train[split_index:]
    y_vali = y_train[split_index:]

    x_train = x_train[:split_index]
    y_train = y_train[:split_index]

    return x_train, y_train, x_vali, y_vali


#==================#
# One-hot embedding
#==================#

def one_hot_coding(y_train, y_test):
    '''
    Encode label to one-hot catogory
    '''
    y_train = to_categorical(y_train, num_classes=2)
    y_test = to_categorical(y_test, num_classes=2)
    return y_train, y_test

def compute_acc(y_ture, y_pred):
    acc = np.sum(y_ture == np.argmax(y_pred, axis=1))/len(y_ture)
    return round(acc, 3)


# CNN model and loss function correction

In [2]:
from tensorflow.keras.applications.resnet50 import ResNet50
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout,BatchNormalization
import tensorflow as tf
import numpy as np



# loss correction
def robust(name, T):

    if name == 'backward':
        T_inv = K.constant(np.linalg.inv(T))

        def loss(y_true, y_pred):
            y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
            y_pred = K.clip(y_pred, K.epsilon(), 1.0 - K.epsilon())
            return -K.sum(K.dot(y_true, T_inv) * K.log(y_pred), axis=-1)

    elif name == 'forward':
        T = K.constant(T)

        def loss(y_true, y_pred):
            y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
            y_pred = K.clip(y_pred, K.epsilon(), 1.0 - K.epsilon())
            
            return -K.sum(y_true * K.log(K.dot(y_pred, T)), axis=-1)

    return loss


# Compute T fucntion
def compute_T(y_predict):
    '''
    model: trained on label noise data
    y_predict: (m,2)
    '''
    T = np.zeros((2, 2))
    index = int(y_predict.shape[0]*0.90)
    x_max_0 = y_predict[y_predict[:,0].argsort()][index]
    x_max_1 = y_predict[y_predict[:,1].argsort()][index]
    T[0][0] = x_max_0[0]
    T[0][1] = x_max_0[1]
    T[1][0] = x_max_1[0]
    T[1][1] = x_max_1[1]
    return T


# model
def train_cnn_model(x_train, y_train, x_vali, y_vali, TYPE='with_noise',
                    lr=0.0001, decay=0.000001, batchsize=256, epochs=1,T=None,loss_name=None):
    '''
    x_train: original data
    y_train: one-hot encoding

    Traning resnet model on label nosie data
    Usesd for estimate the T
    '''
    # data is mnist
    if x_train.shape[3] == 1:
        model = Sequential([
            Conv2D(64, (2, 2), use_bias=True, padding='same', activation='relu',input_shape=(28,28,1)),
            MaxPooling2D(pool_size=(2, 2)),
            Conv2D(64, (2, 2), use_bias=True, padding='same', activation='relu'),
            Flatten(),
            Dense(1000, activation='relu', use_bias=True),
            Dense(500, activation='relu', use_bias=True),
            Dense(2, activation=tf.nn.softmax)
        ])
        
    # data is cifar
    else:
        model = Sequential([
            Conv2D(64, (2, 2), use_bias=True, padding='same', activation='relu',input_shape=(32,32,3)),
            MaxPooling2D(pool_size=(2, 2)),
            Conv2D(64, (2, 2), use_bias=True, padding='same', activation='relu'),
            Flatten(),
            Dense(1000, activation='relu', use_bias=True),
            Dense(500, activation='relu', use_bias=True),
            Dense(2, activation=tf.nn.softmax)
        ])


    datagen = ImageDataGenerator(featurewise_std_normalization=False)
    datagen.fit(x_train)
    adm = optimizers.Adam(lr=lr, decay=decay)
    if TYPE == 'with_noise':
        model.compile(loss='categorical_crossentropy',
                      optimizer=adm,
                      metrics=['accuracy'])
    elif TYPE == 'without_noise':
        model.compile(loss=robust(loss_name, T),
                      optimizer=adm,
                      metrics=['accuracy'])
    history = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batchsize),
                                  steps_per_epoch=len(x_train) / batchsize, epochs=epochs, validation_data=(x_vali, y_vali))
    return model, history

# Benchmark function for SVM, MLP

In [3]:

#--------------#
# SVM benchmark
#--------------#
def SVM_benchmark(x_train, y_train, x_test, y_test, C=1):
    '''
    Do cross validation interation times 
    Data: flatten and normalized 
    Return:
        Accuracy mean and std
    '''
    interation = 10
    train_scores = np.zeros(interation)
    vali_scores = np.zeros(interation)
    test_scores = np.zeros(interation)
    for i in range(interation):
        x_train_, y_train_, x_vali_, y_vali_ = split_data(x_train, y_train)
        clf = svm.SVC(C=C, kernel='poly').fit(x_train_, y_train_)
        train_scores[i] = clf.score(x_train_, y_train_)
        vali_scores[i] = clf.score(x_vali_, y_vali_)
        test_scores[i] = clf.score(x_test, y_test)
    return train_scores.mean(), train_scores.std() * 2, vali_scores.mean(), vali_scores.std() * 2, test_scores.mean(), test_scores.std() * 2



#--------------#
# MLP benchmark
#--------------#
def MLP_benchmark(x_train, y_train, x_test, y_test, layer=(100, 50, 10, 2)):
    '''
    Do cross validation
    Data: flatten and normalized 
    Return:
        Accuracy socre mean and std
    '''
    interation = 10
    train_scores = np.zeros(interation)
    vali_scores = np.zeros(interation)
    test_scores = np.zeros(interation)
    for i in range(interation):
        x_train_, y_train_, x_vali_, y_vali_ = split_data(x_train, y_train)
        clf = MLPClassifier(solver='adam', alpha=1e-5,
                            hidden_layer_sizes=layer).fit(x_train_, y_train_)
        train_scores[i] = clf.score(x_train_, y_train_)
        vali_scores[i] = clf.score(x_vali_, y_vali_)
        test_scores[i] = clf.score(x_test, y_test)
    return train_scores.mean(), train_scores.std() * 2, vali_scores.mean(), vali_scores.std() * 2, test_scores.mean(), test_scores.std() * 2




def cnn_benchmark(x_train, y_train, x_test, y_test,lr=0.0001, decay=0.000001, batchsize=256, epochs=1):
    '''
    x_train is the whole data
    Split trainning data to training data and validation data and feed in model
    
    Return the average trainning acc, vali acc, test acc
    '''
    interation = 10
    train_scores = np.zeros(interation)
    vali_scores = np.zeros(interation)
    test_scores = np.zeros(interation)
    for i in range(interation):
        x_train_, y_train_, x_vali_, y_vali_ = split_data(x_train, y_train)
        
        model, history = train_cnn_model(
            x_train_, y_train_, x_vali_, y_vali_, TYPE='with_noise', 
            lr=lr, decay=decay, batchsize=batchsize, epochs=epochs, T=None,loss_name=None)
        
        train_scores[i] = history.history['acc'][0]
        vali_scores[i] = history.history['val_acc'][0]
        prediction = model.predict(x_test)
        
        test_scores[i] = model.evaluate(x_test, y_test, batch_size=batchsize)[1]
    return train_scores.mean(), train_scores.std() * 2, vali_scores.mean(), vali_scores.std() * 2, test_scores.mean(), test_scores.std() * 2


## MNIST data

In [8]:
# load data a3nd split data
Xtr_mnist, Str_mnist, Xts_mnist, Yts_mnist, Xtr_cifar, Str_cifar, Xts_cifar, Yts_cifar = loaddata()

Str_mnist = Str_mnist.flatten()
Yts_mnist = Yts_mnist.flatten()

Str_cifar = Str_cifar.flatten()
Yts_cifar = Yts_cifar.flatten()


# nomalization training data
Xtr_mnist, Xts_mnist = standard(Xtr_mnist, Xts_mnist)
Xtr_cifar, Xts_cifar = standard(Xtr_cifar, Xts_cifar)


#--------------#
# SVM benchmark
#--------------#
print(SVM_benchmark(Xtr_mnist, Str_mnist, Xts_mnist, Yts_mnist, C=20))


#--------------#
# MLP benchmark
#--------------#
print(MLP_benchmark(Xtr_mnist, Str_mnist, Xts_mnist, Yts_mnist, layer=(100, 50, 10, 2)))


(0.8987999999999999, 0.004344824507388047, 0.6432500000000001, 0.02015068237057991, 0.7979, 0.01214742771124818)
(0.7582625000000001, 0.2669911386638141, 0.61815, 0.046333681053851106, 0.6555500000000001, 0.2682153798722213)


## Cifar

In [9]:
#--------------#
# SVM benchmark
#--------------#

print(SVM_benchmark(Xtr_cifar, Str_cifar, Xts_cifar, Yts_cifar, C=20))

#--------------#
# MLP benchmark
#--------------#
print(MLP_benchmark(Xtr_cifar, Str_cifar, Xts_cifar, Yts_cifar, layer=(100, 50, 10, 2)))

(0.9813875000000001, 0.001660007530103382, 0.605, 0.02261857643619509, 0.67035, 0.015026975743641858)
(0.8232125000000001, 0.25046706495066373, 0.589, 0.040472212689696155, 0.67145, 0.19021695508024514)


# CNN model with loss function correction

## Benchmark and cross validation of CNN without loss function correction on MNIST

In [10]:
#--------------------#
# CNN benchmark for MNIST
#--------------------#
''' '''
Xtr_mnist, Str_mnist, Xts_mnist, Yts_mnist, Xtr_cifar, Str_cifar, Xts_cifar, Yts_cifar = loaddata()

Xtr_mnist,Xts_mnist = standard(Xtr_mnist,Xts_mnist)
Xtr_mnist = Xtr_mnist.reshape(Xtr_mnist.shape[0], 28, 28,1)
Xts_mnist = Xts_mnist.reshape(Xts_mnist.shape[0], 28, 28,1)

y_train, y_test = one_hot_coding(Str_mnist,Yts_mnist)

print('x_train {}, y_train {} \nXts_mnist {}, y_test {}'.format(
    Xtr_mnist.shape,y_train.shape,Xts_mnist.shape,y_test.shape))

result = cnn_benchmark(
     Xtr_mnist, y_train,Xts_mnist,y_test, lr=0.00001, decay=0.0000001, batchsize=1024, epochs=10)


print('{}$\pm${} & {} $\pm${} & {}$\pm${}'.format(round(result[0],3),round(result[1],3),round(result[2],3)
                                                  ,round(result[3],3),result[4],result[5]))

x_train (10000, 28, 28, 1), y_train (10000, 2) 
Xts_mnist (2000, 28, 28, 1), y_test (2000, 2)
Epoch 1/10
8/7 [==============================] - 3s 436ms/step - loss: 0.6760 - acc: 0.5634 - val_loss: 0.6607 - val_acc: 0.6060
Epoch 2/10
8/7 [==============================] - 0s 37ms/step - loss: 0.6517 - acc: 0.6078 - val_loss: 0.6432 - val_acc: 0.6225
Epoch 3/10
8/7 [==============================] - 0s 39ms/step - loss: 0.6362 - acc: 0.6340 - val_loss: 0.6341 - val_acc: 0.6215
Epoch 4/10
8/7 [==============================] - 0s 38ms/step - loss: 0.6260 - acc: 0.6361 - val_loss: 0.6253 - val_acc: 0.6265
Epoch 5/10
8/7 [==============================] - 0s 38ms/step - loss: 0.6176 - acc: 0.6487 - val_loss: 0.6188 - val_acc: 0.6340
Epoch 6/10
8/7 [==============================] - 0s 38ms/step - loss: 0.6114 - acc: 0.6574 - val_loss: 0.6151 - val_acc: 0.6480
Epoch 7/10
8/7 [==============================] - 0s 37ms/step - loss: 0.6073 - acc: 0.6712 - val_loss: 0.6118 - val_acc: 0.6400
Ep

Epoch 1/10
8/7 [==============================] - 1s 145ms/step - loss: 0.6826 - acc: 0.5456 - val_loss: 0.6632 - val_acc: 0.6045
Epoch 2/10
8/7 [==============================] - 0s 35ms/step - loss: 0.6598 - acc: 0.5994 - val_loss: 0.6508 - val_acc: 0.6045
Epoch 3/10
8/7 [==============================] - 0s 34ms/step - loss: 0.6467 - acc: 0.6104 - val_loss: 0.6407 - val_acc: 0.6340
Epoch 4/10
8/7 [==============================] - 0s 35ms/step - loss: 0.6362 - acc: 0.6332 - val_loss: 0.6331 - val_acc: 0.6365
Epoch 5/10
8/7 [==============================] - 0s 38ms/step - loss: 0.6274 - acc: 0.6357 - val_loss: 0.6262 - val_acc: 0.6395
Epoch 6/10
8/7 [==============================] - 0s 41ms/step - loss: 0.6198 - acc: 0.6410 - val_loss: 0.6217 - val_acc: 0.6365
Epoch 7/10
8/7 [==============================] - 0s 34ms/step - loss: 0.6134 - acc: 0.6543 - val_loss: 0.6179 - val_acc: 0.6400
Epoch 8/10
8/7 [==============================] - 0s 41ms/step - loss: 0.6085 - acc: 0.6560 - va

## Benchmark and cross validation of CNN without loss function correction on Cifar

In [11]:
#--------------------#
# CNN benchmark for cifa
#--------------------#

Xtr_cifar,Xts_cifar = standard(Xtr_cifar,Xts_cifar)

Xtr_cifar = Xtr_cifar.reshape(Xtr_cifar.shape[0], 32,32,3)
Xts_cifar = Xts_cifar.reshape(Xts_cifar.shape[0], 32,32,3)
y_train, y_test = one_hot_coding(Str_cifar,Yts_cifar)

print('x_train {}, y_train {} \nXts_mnist {}, y_test {}'.format(
    Xtr_cifar.shape,y_train.shape,Xts_cifar.shape,y_test.shape))

result = cnn_benchmark(
     Xtr_cifar, y_train,Xts_cifar,y_test, lr=0.00005, decay=0.0000001, batchsize=1024, epochs=10)


print('{}$\pm${} & {} $\pm${} & {}$\pm${}'.format(round(result[0],3),round(result[1],3),round(result[2],3)
                                                  ,round(result[3],3),result[4],result[5]))

x_train (10000, 32, 32, 3), y_train (10000, 2) 
Xts_mnist (2000, 32, 32, 3), y_test (2000, 2)
Epoch 1/10
8/7 [==============================] - 2s 238ms/step - loss: 0.6888 - acc: 0.5791 - val_loss: 0.6698 - val_acc: 0.6030
Epoch 2/10
8/7 [==============================] - 0s 47ms/step - loss: 0.6488 - acc: 0.6278 - val_loss: 0.6438 - val_acc: 0.6140
Epoch 3/10
8/7 [==============================] - 0s 46ms/step - loss: 0.6330 - acc: 0.6446 - val_loss: 0.6372 - val_acc: 0.6430
Epoch 4/10
8/7 [==============================] - 0s 47ms/step - loss: 0.6210 - acc: 0.6553 - val_loss: 0.6375 - val_acc: 0.6460
Epoch 5/10
8/7 [==============================] - 0s 47ms/step - loss: 0.6146 - acc: 0.6715 - val_loss: 0.6346 - val_acc: 0.6450
Epoch 6/10
8/7 [==============================] - 0s 51ms/step - loss: 0.6098 - acc: 0.6713 - val_loss: 0.6334 - val_acc: 0.6475
Epoch 7/10
8/7 [==============================] - 0s 49ms/step - loss: 0.6025 - acc: 0.6814 - val_loss: 0.6304 - val_acc: 0.6450
Ep

Epoch 1/10
8/7 [==============================] - 2s 226ms/step - loss: 0.6586 - acc: 0.6092 - val_loss: 0.6289 - val_acc: 0.6640
Epoch 2/10
8/7 [==============================] - 0s 47ms/step - loss: 0.6292 - acc: 0.6523 - val_loss: 0.6139 - val_acc: 0.6485
Epoch 3/10
8/7 [==============================] - 0s 46ms/step - loss: 0.6145 - acc: 0.6657 - val_loss: 0.6185 - val_acc: 0.6630
Epoch 4/10
8/7 [==============================] - 0s 46ms/step - loss: 0.6044 - acc: 0.6820 - val_loss: 0.6106 - val_acc: 0.6640
Epoch 5/10
8/7 [==============================] - 0s 47ms/step - loss: 0.5929 - acc: 0.6910 - val_loss: 0.6122 - val_acc: 0.6650
Epoch 6/10
8/7 [==============================] - 0s 48ms/step - loss: 0.5819 - acc: 0.7029 - val_loss: 0.6137 - val_acc: 0.6590
Epoch 7/10
8/7 [==============================] - 0s 56ms/step - loss: 0.5721 - acc: 0.7120 - val_loss: 0.6196 - val_acc: 0.6615
Epoch 8/10
8/7 [==============================] - 0s 46ms/step - loss: 0.5603 - acc: 0.7280 - va

# Compute matrix T (flip rate)

## Utility function

In [5]:
def cnn_loss_correct_compute_T(x_train,y_train,x_test,y_test,iteration=10,lr=0.00005, decay=0.0000001, 
                     batchsize=1024,epochs=15,TYPE='mnist',loss_name='forward'):
    if TYPE=='mnist':
        x_train = x_train.reshape(x_train.shape[0], 28,28,1)
        x_test = x_test.reshape(x_test.shape[0], 28,28,1)
        y_train, y_test = one_hot_coding(y_train,y_test)
    else:
        x_train = x_train.reshape(x_train.shape[0], 32,32,3)
        x_test = x_test.reshape(x_test.shape[0], 32,32,3)
        y_train, y_test = one_hot_coding(y_train,y_test)
        
    x_train_= x_train.copy()
    y_train_=y_train.copy()
    
    T = np.zeros((iteration,2,2))
    
    for i in range(iteration):
        
        x_train, y_train, x_vali, y_vali = split_data(x_train_,y_train_,part=0.8)
        
        print('\nIteration {}\n'.format(i))
        
        # Estimate the T
        print('\nTrain model with noise \n')
        model_noise,history_noise = train_cnn_model(x_train, y_train, x_vali, y_vali, TYPE='with_noise',
                            lr=lr, decay=decay, batchsize=batchsize, epochs=epochs)
        
        prediction_noise = model_noise.predict(x_train)
        T[i] = compute_T(prediction_noise)
    
    # Return the average of T
    return (np.mean(T,axis=0),
            np.std(T,axis=0))

# Implement the given flip rate to train model 
def cnn_loss_correct(x_train,y_train,x_test,y_test,iteration=10,lr=0.00005, decay=0.0000001, 
                     batchsize=1024,epochs=15,TYPE='mnist',loss_name='forward'):
    if TYPE=='mnist':
        x_train = x_train.reshape(x_train.shape[0], 28,28,1)
        x_test = x_test.reshape(x_test.shape[0], 28,28,1)
        y_train, y_test = one_hot_coding(y_train,y_test)
    else:
        x_train = x_train.reshape(x_train.shape[0], 32,32,3)
        x_test = x_test.reshape(x_test.shape[0], 32,32,3)
        y_train, y_test = one_hot_coding(y_train,y_test)
        
    x_train_= x_train.copy()
    y_train_=y_train.copy()
    
    T = np.zeros((iteration,2,2))
    
    no_noise_accuracy = np.zeros((iteration))
    validation_accuracy = np.zeros((iteration))
    train_accuracy = np.zeros((iteration))
    for i in range(iteration):
        
        x_train, y_train, x_vali, y_vali = split_data(x_train_,y_train_,part=0.8)
        
        print('\nIteration {}\n'.format(i))
        
        print('\nTrain model with loss orrection \n')
        
        # Given the T (flip rate) 
        T = np.array([[0.8,0.2],[0.4,0.6]])
        model, history = train_cnn_model(x_train, y_train, x_vali, y_vali, TYPE='without_noise',
                            lr=lr, decay=decay, batchsize=batchsize, epochs=epochs,T=T,loss_name=loss_name)
        
        train_accuracy[i] = model.evaluate(x_train, y_train)[1]
        validation_accuracy[i] = model.evaluate(x_vali, y_vali)[1]
        no_noise_accuracy[i] = model.evaluate(x_test,y_test)[1]
        
    model.summary()
    return (np.mean(train_accuracy),train_accuracy.std(),
            np.mean(validation_accuracy),validation_accuracy.std(),
            np.mean(no_noise_accuracy),no_noise_accuracy.std(),
            np.mean(T,axis=0),
            np.std(T,axis=0))

## On MNIST

### Compute T

In [13]:
Xtr_mnist, Str_mnist, Xts_mnist, Yts_mnist, Xtr_cifar, Str_cifar, Xts_cifar, Yts_cifar = loaddata()
Xtr_mnist,Xts_mnist = standard(Xtr_mnist,Xts_mnist)

result = cnn_loss_correct_compute_T(Xtr_mnist,Str_mnist,Xts_mnist,Yts_mnist,iteration=10,lr=0.00001, decay=0.0000001, 
                     batchsize=1024,epochs=10,TYPE='mnist',loss_name = 'forward')


Iteration 0


Train model with noise 

Epoch 1/10
8/7 [==============================] - 2s 226ms/step - loss: 0.6795 - acc: 0.5592 - val_loss: 0.6642 - val_acc: 0.5965
Epoch 2/10
8/7 [==============================] - 0s 36ms/step - loss: 0.6485 - acc: 0.6184 - val_loss: 0.6488 - val_acc: 0.6215
Epoch 3/10
8/7 [==============================] - 0s 38ms/step - loss: 0.6351 - acc: 0.6342 - val_loss: 0.6373 - val_acc: 0.6225
Epoch 4/10
8/7 [==============================] - 0s 38ms/step - loss: 0.6246 - acc: 0.6377 - val_loss: 0.6261 - val_acc: 0.6235
Epoch 5/10
8/7 [==============================] - 0s 38ms/step - loss: 0.6174 - acc: 0.6445 - val_loss: 0.6189 - val_acc: 0.6495
Epoch 6/10
8/7 [==============================] - 0s 37ms/step - loss: 0.6134 - acc: 0.6541 - val_loss: 0.6159 - val_acc: 0.6420
Epoch 7/10
8/7 [==============================] - 0s 37ms/step - loss: 0.6095 - acc: 0.6649 - val_loss: 0.6123 - val_acc: 0.6585
Epoch 8/10
8/7 [==============================] - 0s 38m

TypeError: type numpy.ndarray doesn't define __round__ method

In [19]:
# Print the average T and std
print('{}  \n \n  {}'.format(result[0],result[1]))

[[0.81990661 0.18009339]
 [0.40075063 0.59924937]]  
 
  [[0.01070622 0.01070623]
 [0.01391703 0.01391703]]


### Use given T to train and test model

In [6]:
Xtr_mnist, Str_mnist, Xts_mnist, Yts_mnist, Xtr_cifar, Str_cifar, Xts_cifar, Yts_cifar = loaddata()

Xtr_mnist,Xts_mnist = standard(Xtr_mnist,Xts_mnist)


result = cnn_loss_correct(Xtr_mnist, Str_mnist, Xts_mnist, Yts_mnist,iteration=10,lr=0.00005, decay=0.0000001, 
                     batchsize=1024,epochs=10,TYPE='mnist',loss_name = 'forward')

# Print the test accuracy
print('{}$\pm${}'.format(round(result[4],3),round(result[5],3)))


Iteration 0


Train model with loss orrection 

Epoch 1/10
8/7 [==============================] - 2s 263ms/step - loss: 0.6569 - acc: 0.5999 - val_loss: 0.6381 - val_acc: 0.6560
Epoch 2/10
8/7 [==============================] - 0s 34ms/step - loss: 0.6288 - acc: 0.6677 - val_loss: 0.6195 - val_acc: 0.6635
Epoch 3/10
8/7 [==============================] - 0s 34ms/step - loss: 0.6140 - acc: 0.6713 - val_loss: 0.6138 - val_acc: 0.6470
Epoch 4/10
8/7 [==============================] - 0s 33ms/step - loss: 0.6080 - acc: 0.6699 - val_loss: 0.6045 - val_acc: 0.6675
Epoch 5/10
8/7 [==============================] - 0s 43ms/step - loss: 0.6040 - acc: 0.6706 - val_loss: 0.6031 - val_acc: 0.6745
Epoch 6/10
8/7 [==============================] - 0s 34ms/step - loss: 0.6006 - acc: 0.6785 - val_loss: 0.6009 - val_acc: 0.6735
Epoch 7/10
8/7 [==============================] - 0s 39ms/step - loss: 0.5987 - acc: 0.6831 - val_loss: 0.5999 - val_acc: 0.6775
Epoch 8/10
8/7 [==============================]

8/7 [==============================] - 0s 34ms/step - loss: 0.6047 - acc: 0.6769 - val_loss: 0.5944 - val_acc: 0.6760
Epoch 6/10
8/7 [==============================] - 0s 41ms/step - loss: 0.6031 - acc: 0.6733 - val_loss: 0.5949 - val_acc: 0.6660
Epoch 7/10
8/7 [==============================] - 0s 35ms/step - loss: 0.6009 - acc: 0.6795 - val_loss: 0.5931 - val_acc: 0.6705
Epoch 8/10
8/7 [==============================] - 0s 41ms/step - loss: 0.5990 - acc: 0.6839 - val_loss: 0.5914 - val_acc: 0.6895
Epoch 9/10
8/7 [==============================] - 0s 34ms/step - loss: 0.5976 - acc: 0.6827 - val_loss: 0.5907 - val_acc: 0.6875
Epoch 10/10
2000/2000 [==============================] - 0s 68us/step

Iteration 6


Train model with loss orrection 

Epoch 1/10
8/7 [==============================] - 1s 165ms/step - loss: 0.6550 - acc: 0.6050 - val_loss: 0.6353 - val_acc: 0.6595
Epoch 2/10
8/7 [==============================] - 0s 34ms/step - loss: 0.6286 - acc: 0.6582 - val_loss: 0.6162 - val_

## Cifar

### Compute T 

In [7]:
Xtr_mnist, Str_mnist, Xts_mnist, Yts_mnist, Xtr_cifar, Str_cifar, Xts_cifar, Yts_cifar = loaddata()

Xtr_cifar,Xts_cifar = standard(Xtr_cifar,Xts_cifar)

result = cnn_loss_correct_compute_T(Xtr_cifar, Str_cifar, Xts_cifar, Yts_cifar,iteration=10,lr=0.00005, decay=0.0000001, 
                     batchsize=1024,epochs=10,TYPE='cifar',loss_name = 'forward')

# Print the average T and std
print('{}  \n \n  {}'.format(result[0],result[1]))


Iteration 0


Train model with noise 

Epoch 1/10
8/7 [==============================] - 2s 232ms/step - loss: 0.6792 - acc: 0.5760 - val_loss: 0.6533 - val_acc: 0.6205
Epoch 2/10
8/7 [==============================] - 0s 50ms/step - loss: 0.6427 - acc: 0.6343 - val_loss: 0.6328 - val_acc: 0.6410
Epoch 3/10
8/7 [==============================] - 0s 45ms/step - loss: 0.6272 - acc: 0.6480 - val_loss: 0.6284 - val_acc: 0.6480
Epoch 4/10
8/7 [==============================] - 0s 46ms/step - loss: 0.6182 - acc: 0.6655 - val_loss: 0.6243 - val_acc: 0.6625
Epoch 5/10
8/7 [==============================] - 0s 46ms/step - loss: 0.6107 - acc: 0.6752 - val_loss: 0.6235 - val_acc: 0.6570
Epoch 6/10
8/7 [==============================] - 0s 48ms/step - loss: 0.6014 - acc: 0.6848 - val_loss: 0.6203 - val_acc: 0.6595
Epoch 7/10
8/7 [==============================] - 0s 46ms/step - loss: 0.5931 - acc: 0.6980 - val_loss: 0.6226 - val_acc: 0.6610
Epoch 8/10
8/7 [==============================] - 0s 49m

8/7 [==============================] - 0s 46ms/step - loss: 0.6427 - acc: 0.6283 - val_loss: 0.6311 - val_acc: 0.6650
Epoch 3/10
8/7 [==============================] - 0s 45ms/step - loss: 0.6273 - acc: 0.6512 - val_loss: 0.6293 - val_acc: 0.6635
Epoch 4/10
8/7 [==============================] - 0s 54ms/step - loss: 0.6199 - acc: 0.6638 - val_loss: 0.6240 - val_acc: 0.6630
Epoch 5/10
8/7 [==============================] - 0s 46ms/step - loss: 0.6101 - acc: 0.6776 - val_loss: 0.6219 - val_acc: 0.6700
Epoch 6/10
8/7 [==============================] - 0s 45ms/step - loss: 0.6037 - acc: 0.6795 - val_loss: 0.6223 - val_acc: 0.6670
Epoch 7/10
8/7 [==============================] - 0s 45ms/step - loss: 0.5953 - acc: 0.6903 - val_loss: 0.6212 - val_acc: 0.6660
Epoch 8/10
8/7 [==============================] - 0s 47ms/step - loss: 0.5863 - acc: 0.6998 - val_loss: 0.6208 - val_acc: 0.6660
Epoch 9/10
8/7 [==============================] - 0s 53ms/step - loss: 0.5769 - acc: 0.7111 - val_loss: 0.62

### Use given T to train and test model

In [8]:
Xtr_mnist, Str_mnist, Xts_mnist, Yts_mnist, Xtr_cifar, Str_cifar, Xts_cifar, Yts_cifar = loaddata()

Xtr_cifar,Xts_cifar = standard(Xtr_cifar,Xts_cifar)

result = cnn_loss_correct(Xtr_cifar,Str_cifar,Xts_cifar,Yts_cifar,iteration=10,lr=0.00005, decay=0.0000001, 
                     batchsize=1024,epochs=10,TYPE='cifar',loss_name = 'forward')

# Print the test accuracy
print('{}$\pm${}'.format(round(result[4],3),round(result[5],3)))


Iteration 0


Train model with loss orrection 

Epoch 1/10
8/7 [==============================] - 2s 245ms/step - loss: 0.6599 - acc: 0.5438 - val_loss: 0.6415 - val_acc: 0.5770
Epoch 2/10
8/7 [==============================] - 0s 45ms/step - loss: 0.6358 - acc: 0.6088 - val_loss: 0.6284 - val_acc: 0.6340
Epoch 3/10
8/7 [==============================] - 0s 46ms/step - loss: 0.6255 - acc: 0.6286 - val_loss: 0.6241 - val_acc: 0.6340
Epoch 4/10
8/7 [==============================] - 0s 46ms/step - loss: 0.6193 - acc: 0.6400 - val_loss: 0.6228 - val_acc: 0.6490
Epoch 5/10
8/7 [==============================] - 0s 46ms/step - loss: 0.6157 - acc: 0.6496 - val_loss: 0.6205 - val_acc: 0.6425
Epoch 6/10
8/7 [==============================] - 0s 46ms/step - loss: 0.6099 - acc: 0.6570 - val_loss: 0.6189 - val_acc: 0.6475
Epoch 7/10
8/7 [==============================] - 0s 53ms/step - loss: 0.6058 - acc: 0.6665 - val_loss: 0.6179 - val_acc: 0.6500
Epoch 8/10
8/7 [==============================]

8/7 [==============================] - 0s 47ms/step - loss: 0.6129 - acc: 0.6564 - val_loss: 0.6245 - val_acc: 0.6310
Epoch 6/10
8/7 [==============================] - 0s 51ms/step - loss: 0.6095 - acc: 0.6601 - val_loss: 0.6236 - val_acc: 0.6360
Epoch 7/10
8/7 [==============================] - 0s 49ms/step - loss: 0.6082 - acc: 0.6577 - val_loss: 0.6236 - val_acc: 0.6425
Epoch 8/10
8/7 [==============================] - 0s 46ms/step - loss: 0.6020 - acc: 0.6787 - val_loss: 0.6250 - val_acc: 0.6430
Epoch 9/10
8/7 [==============================] - 0s 46ms/step - loss: 0.5965 - acc: 0.6854 - val_loss: 0.6218 - val_acc: 0.6425
Epoch 10/10
2000/2000 [==============================] - 0s 100us/step

Iteration 6


Train model with loss orrection 

Epoch 1/10
8/7 [==============================] - 2s 280ms/step - loss: 0.6564 - acc: 0.5492 - val_loss: 0.6400 - val_acc: 0.6055
Epoch 2/10
8/7 [==============================] - 0s 46ms/step - loss: 0.6337 - acc: 0.6165 - val_loss: 0.6278 - val